In [1]:
import numpy as np
import pandas as pd
import itertools as it
import pdb

In [2]:
%load_ext line_profiler

In [3]:
df_A = pd.read_excel("Deudas.xlsx")
df_A

,Documento,Nombre,Deuda
0,101,Alice,21
1,102,Bob,123
2,103,Alice,84
3,104,Carol,49


In [4]:
df_B = pd.read_excel("Pagos.xlsx")
df_B

,Documento,Nombre,Pago
0,201,Bob,59
1,202,Alice,49
2,203,Carol,21
3,204,Alice,21
4,205,Bob,64
5,206,Bob,27
6,207,Alice,14
7,208,Carol,12
8,209,Alice,15
9,210,Carol,37


## Uno-a-muchos

In [5]:
# NOTA: Aquí el mismo documento de pago puede ser pareado en distintos calces.
# Esto se debe arreglar
def funcion(fila, grupos):
    seleccion = grupos.get_group(fila["Nombre"])
    for largo in range(1, seleccion.size + 1):
        for comb in it.combinations(seleccion.index, largo):
            subseleccion = seleccion.loc[list(comb)]
            if fila["Deuda"] == subseleccion["Pago"].sum():
                resultado = pd.merge(pd.DataFrame(fila).T, subseleccion, on="Nombre")
                return resultado
    return pd.DataFrame(columns=seleccion.columns)

In [6]:
grupos = df_B.groupby("Nombre")
pd.concat(list(df_A.apply(funcion, args=(grupos,), axis=1)))

,Documento_x,Nombre,Deuda,Documento_y,Pago
0,101,Alice,21,204,21
0,102,Bob,123,201,59
1,102,Bob,123,205,64
0,103,Alice,84,202,49
1,103,Alice,84,204,21
2,103,Alice,84,207,14
0,104,Carol,49,203,21
1,104,Carol,49,211,28


In [7]:
%timeit pd.concat(list(df_A.apply(funcion, args=(grupos,), axis=1)))

20.5 ms ± 653 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%lprun -f funcion funcion(df_A.iloc[1], grupos)

## Uno-a-uno, luego uno-a-todos y después uno-a-muchos (EN PROCESO)

In [9]:
def procesar(df_A, df_B):
    merge = pd.merge(df_A, df_B, on="Nombre")
    
    # Uno-a-uno
    match_1 = merge[merge["Deuda"] == merge["Pago"]]
    match_1 = match_1.drop_duplicates(subset="Documento_y")
    
    merge = merge[(~merge["Documento_x"].isin(match_1["Documento_x"])) & (~merge["Documento_y"].isin(match_1["Documento_y"]))]
    
    # Uno-a-todos
    # <código>
    
    # Uno-a-muchos
    # <código>
    
    return match_1

In [10]:
procesar(df_A, df_B)

,Documento_x,Nombre,Deuda,Documento_y,Pago
1,101,Alice,21,204,21


In [11]:
%lprun -f procesar procesar(df_A, df_B)